Following guide in https://github.com/ekapope/WorldWeatherOnline/tree/master/wwo_hist

# Text Scrapping Assignment
Precipitation is a major driver for non-point source pollutants runoff, and the impact of precipitation may vary annually, seasonally, or even monthly. For example, soil is more prone to erosion during planting seasons (March-May) when land cover is minimal. Consequently, a similar precipitation events (intensity and duration) may result in greater soil erosion during planting season as compared to the late-growing season. <br>

Therefore, it is important to understand the precipitation pattern when evaluating the fate and transport of non-point source pollutants. The objectives of this task are: <br>
1) To gather historical precipitation data through an open API <br>
2) Calculate 30-year average annual precipitation depth <br>
3) Calculate annual precipitation depth for the monitoring years (2016-2018) <br>
4) Compare annual precipitation depth to the 30-year average, then determine if it is a dry, normal, or wet year <br>
5) Calculate 30-year average monthly precipitation depth <br>
6) Calculate monthly precipitation depth, then determine if it is a dry, normal, or wet month <br>

In the future development, this data will be paired with nutrient concentration and loading data to identify any possible correlations.


In [1]:
import urllib
import urllib.parse
import json
import pandas as pd
import datetime as dt
import calendar
from datetime import datetime

pd.set_option('mode.chained_assignment', None)

In [2]:
def extract_monthly_data(data):
    num_days = len(data)
    # initialize df_month to store return data
    df_month = pd.DataFrame()
    for i in range(num_days):
        # extract this day
        d = data[i]
        # astronomy data is the same for the whole day
        astr_df = pd.DataFrame(d['astronomy'])
        # hourly data; temperature for each hour of the day
        hourly_df = pd.DataFrame(d['hourly'])
        # this wanted_key will be duplicated and use 'ffill' to fill up the NAs
        wanted_keys = ['date', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex'] # The keys you want
        subset_d = dict((k, d[k]) for k in wanted_keys if k in d)
        this_df = pd.DataFrame(subset_d,index=[0])        
        df = pd.concat([this_df.reset_index(drop=True), astr_df], axis=1)
        # concat selected astonomy columns with hourly data
        df = pd.concat([df,hourly_df], axis=1)
        df = df.fillna(method='ffill')
        # make date_time columm to proper format
        # fill leading zero for hours to 4 digits (0000-2400 hr)
        df['time'] = df['time'].apply(lambda x: x.zfill(4))
        # keep only first 2 digit (00-24 hr) 
        df['time'] = df['time'].str[:2]
        # convert to pandas datetime
        df['date_time'] = pd.to_datetime(df['date'] + ' ' + df['time'])
        # keep only interested columns
        col_to_keep = ['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex', 
               'moon_illumination', 'moonrise', 'moonset', 'sunrise', 'sunset', 
               'DewPointC',  'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph',
               'cloudcover', 'humidity', 'precipMM', 'pressure', 'tempC', 'visibility',
               'winddirDegree', 'windspeedKmph']
        df = df[col_to_keep]
        df_month = pd.concat([df_month,df])
    return(df_month)

In [3]:
#function to retrive data by date range and location
#default frequency = 1 hr
#each month costs 1 request (free trial 500 requests/key, as of 30-May-2019)
def retrieve_this_location(api_key,location,start_date,end_date,frequency):
    
    start_time = datetime.now()
    
    # create list of months, convert to month begins (first day of each month)
    list_mon_begin= pd.date_range(start_date,end_date, freq='1M')-pd.offsets.MonthBegin(1)
    # convert to Series and append first day of the last month
    list_mon_begin = pd.concat([pd.Series(list_mon_begin), pd.Series(pd.to_datetime(end_date,infer_datetime_format=True).replace(day=1))], ignore_index=True)
    # change the begin date to start_date
    list_mon_begin[0] = pd.to_datetime(start_date,infer_datetime_format=True)
    
    # create list of months, convert to month ends (last day of each month)
    list_mon_end = pd.date_range(start_date,end_date, freq='1M')-pd.offsets.MonthEnd(0)
    # convert to Series and append the end_date
    list_mon_end = pd.concat([pd.Series(list_mon_end), pd.Series(pd.to_datetime(end_date,infer_datetime_format=True))], ignore_index=True)
    
    # count number of months to be retrieved
    total_months = len(list_mon_begin)

    # initialize df_hist to store return data
    df_hist = pd.DataFrame()
    for m in range(total_months):
        
        start_d =str(list_mon_begin[m])[:10]
        end_d =str(list_mon_end[m])[:10]
        print('Currently retrieving data for '+location+': from '+start_d+' to '+end_d)
        
        url_page = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key='+api_key+'&q='+location+'&format=json&date='+start_d+'&enddate='+end_d+'&tp='+str(frequency)
        json_page = urllib.request.urlopen(url_page)
        json_data = json.loads(json_page.read().decode())
        data= json_data['data']['weather']
       # call function to extract json object
        df_this_month = extract_monthly_data(data)
        df_hist = pd.concat([df_hist,df_this_month])
        
        time_elapsed = datetime.now() - start_time
        print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
    return(df_hist)



In [4]:
# retrieving data for dates provided by user

API_key = '3bbd19dc238c4542a08213223192409'
location = '42.2422,-95.0381'
startdate = input('Enter start date as YYYY-MM-DD')
enddate = input('Enter end date as YYYY-MM-DD')
startdate = datetime.strptime(startdate, '%Y-%m-%d')
enddate = datetime.strptime(enddate, '%Y-%m-%d')
weather_data  = retrieve_this_location(API_key, location, startdate, enddate,'default')

Enter start date as YYYY-MM-DD2016-01-01
Enter end date as YYYY-MM-DD2018-12-31
Currently retrieving data for 42.2422,-95.0381: from 2016-01-01 to 2016-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.297995
Currently retrieving data for 42.2422,-95.0381: from 2016-02-01 to 2016-02-29
Time elapsed (hh:mm:ss.ms) 0:00:02.507604
Currently retrieving data for 42.2422,-95.0381: from 2016-03-01 to 2016-03-31
Time elapsed (hh:mm:ss.ms) 0:00:03.715392
Currently retrieving data for 42.2422,-95.0381: from 2016-04-01 to 2016-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.879457
Currently retrieving data for 42.2422,-95.0381: from 2016-05-01 to 2016-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.087176
Currently retrieving data for 42.2422,-95.0381: from 2016-06-01 to 2016-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.306476
Currently retrieving data for 42.2422,-95.0381: from 2016-07-01 to 2016-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.787652
Currently retrieving data for 42.2422,-95.0381: from 2016-08-01 to 2016-08-31
Ti

In [5]:
weather_data.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex,moon_illumination,moonrise,moonset,...,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2016-01-01 00:00:00,-2,-9,0.0,8.7,1,0,50,12:36 AM,12:53 PM,...,-16,34,98,93,0.0,1030,-8,10,288,22
1,2016-01-01 03:00:00,-2,-9,0.0,8.7,1,0,50,12:36 AM,12:53 PM,...,-16,40,92,94,0.0,1029,-8,10,286,27
2,2016-01-01 06:00:00,-2,-9,0.0,8.7,1,0,50,12:36 AM,12:53 PM,...,-18,51,23,90,0.0,1028,-9,10,272,29
3,2016-01-01 09:00:00,-2,-9,0.0,8.7,1,2,50,12:36 AM,12:53 PM,...,-15,56,8,81,0.0,1026,-6,10,268,30
4,2016-01-01 12:00:00,-2,-9,0.0,8.7,1,2,50,12:36 AM,12:53 PM,...,-9,41,16,79,0.0,1024,-2,10,246,26


In [6]:
ppt =  weather_data [['date_time','precipMM','tempC']]
ppt.head()

,date_time,precipMM,tempC
0,2016-01-01 00:00:00,0.0,-8
1,2016-01-01 03:00:00,0.0,-8
2,2016-01-01 06:00:00,0.0,-9
3,2016-01-01 09:00:00,0.0,-6
4,2016-01-01 12:00:00,0.0,-2


In [8]:
# separate date and time so that daily average can be calculate using "groupby"
ppt['Date']= ppt['date_time'].dt.date

In [9]:
print(ppt.head())
print(ppt.dtypes)

            date_time precipMM tempC        Date
0 2016-01-01 00:00:00      0.0    -8  2016-01-01
1 2016-01-01 03:00:00      0.0    -8  2016-01-01
2 2016-01-01 06:00:00      0.0    -9  2016-01-01
3 2016-01-01 09:00:00      0.0    -6  2016-01-01
4 2016-01-01 12:00:00      0.0    -2  2016-01-01
date_time    datetime64[ns]
precipMM             object
tempC                object
Date                 object
dtype: object


In [10]:
# converting precip and temperature values from object to string then to interger/float
ppt['Date'] = pd.to_datetime(ppt['Date'])
ppt['precipMM'] = ppt['precipMM'].astype(float)
ppt['tempC'] = ppt['tempC'].astype(str).astype(int)

In [75]:
# calculate daily cumulative precip and average temp
daily_ppt = ppt.groupby('Date').agg({'precipMM':'sum', 'tempC':'mean'})
print(daily_ppt.head())
print(daily_ppt.dtypes)

            precipMM   tempC
Date                        
2016-01-01       0.0  -5.750
2016-01-02       0.0  -6.000
2016-01-03       0.1  -6.625
2016-01-04       0.1 -10.125
2016-01-05       0.0  -4.875
precipMM    float64
tempC       float64
dtype: object


In [78]:
export_ppt = daily_ppt.to_csv('BHL_weather_data.csv', index=True)

In [84]:
startyear = startdate.year
endyear = enddate.year

years = range(startyear, endyear+1)
start, end = startdate, enddate + dt.timedelta(1)

# first For loop is to calculate number of days in each year of interest
y = startyear #year counter
for year in years:
    year_start = dt.datetime(year, 1, 1)
    year_end = dt.datetime(year+1, 1, 1)
    number_of_days = [(min(end, year_end) - max(start, year_start)).days]
    
    # second For loop to calculate cumulative precipitation in the year
    ### 9/27/19 update: I should first slice the df into annual sets (i.e. 2016 one set, 2017 one set)
    ### Google "python using for loop to calculate cumulative value new column" 
    cum_precip = 0
    for day in number_of_days:
        cum_precip = cum_precip + daily_ppt['precipMM']
        print(cum_precip)
        #print(number_of_days)
        
    y = y + 1
    #print(y, number_of_days)

Date
2016-01-01    0.0
2016-01-02    0.0
2016-01-03    0.1
2016-01-04    0.1
2016-01-05    0.0
2016-01-06    0.4
2016-01-07    0.9
2016-01-08    0.4
2016-01-09    0.0
2016-01-10    0.1
2016-01-11    0.1
2016-01-12    0.0
2016-01-13    0.0
2016-01-14    0.0
2016-01-15    0.2
2016-01-16    0.0
2016-01-17    0.0
2016-01-18    0.0
2016-01-19    0.7
2016-01-20    0.0
2016-01-21    0.0
2016-01-22    0.3
2016-01-23    0.1
2016-01-24    0.1
2016-01-25    0.5
2016-01-26    0.0
2016-01-27    0.0
2016-01-28    0.0
2016-01-29    0.0
2016-01-30    0.0
             ... 
2018-12-02    3.4
2018-12-03    0.2
2018-12-04    0.2
2018-12-05    0.0
2018-12-06    0.0
2018-12-07    0.2
2018-12-08    0.2
2018-12-09    0.0
2018-12-10    0.0
2018-12-11    0.0
2018-12-12    0.0
2018-12-13    0.0
2018-12-14    0.0
2018-12-15    0.0
2018-12-16    0.0
2018-12-17    0.0
2018-12-18    0.0
2018-12-19    0.0
2018-12-20    0.0
2018-12-21    0.0
2018-12-22    0.0
2018-12-23    0.0
2018-12-24    0.0
2018-12-25    0.0
2018-